## Naive Bayes Text Classification ##

Tha following program applies the Naive Bayes classifier provided by NLTK to input data files.

In [ ]:
# Author: Elizabeth Brooks
# Date Modified: 07/06/2015

# PreProcessor Directives
%matplotlib inline
import matplotlib
import csv
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from scipy.misc import imread
import re
# Import necessary NB classifier func
from nltk.classify import apply_features
import random
# Add parent directory to path for twc imports
import os
import sys
sys.path.append(os.path.realpath('../'))
import twittercriteria as twc

# Global field declarations
twc.loadCriteria()
twc.clearCriteria()
current_dir = os.getcwd()

# Function to clean up tweet strings 
# by manually removing irrelevant data (not words)
def cleanUpTweet(tweet_text):
    # Irrelevant characters
    twitterMarkup = ['&amp;', 'http://t.co/']
    temp = tweet_text.lower()
    # Use regex to create a regular expression 
    # for removing undesired characters
    temp = re.sub('|'.join(twitterMarkup), r"", temp)
    return temp
# End cleanUpTweet

# Function to search for tweets based on pre-determined key words
def tweetHasAKeyword(tweet_text):
    return keyword.search(tweet_text) is not None
# End tweetHasAKeyword

The above function removes irrelevant characters from the tweet strings contained in the Twitter data csv files.

In [ ]:
# Function for organizing data for labeling
def orgData(txt_data):
    # Set the output file path
    relevantPath = current_dir + '/RelevantTweets.txt'
    irrelevantPath = current_dir + '/IrrelevantTweets.txt'
    # Create object for writting to a text file
    relevantTxtFile = open(relevantPath, "w")
    irrelevantTxtFile = open(irrelevantPath, "w")
    # Determie the label to assign a tweet based on keyword
    if tweetHasAKeyword(tweetText):
        relevantTxtFile.write(tweetText)
    else:
        irrelevantTxtFile.write(tweetText)
    # End else
    # Close the files
    relevantTxtFile.close()
    irrelevantTxtFile.close()
# End orgData

# Function to extract relevant features
def extractFeatures(txt_data):
    # Create object for writting to a text file
    relevantTxtFile = open(relevantPath, "w")
    irrelevantTxtFile = open(irrelevantPath, "w")
    # Assign labels to tweets
    labeledTweets = ([(word, 'relevant') for word in relevantTxtFile.read().split()] +
        [(word, 'irrelevant') for word in irrelevantTxtFile.read().split()])
    # Close the files
    relevantTxtFile.close()
    irrelevantTxtFile.close()
    
    # Extract keywords
# End trainTxtData


The above functions are used to first organize the data into text file by relevance accoring to the pre determined list of keyword, then a second function extracts these tweets features to be used to build classes and identify tweets for classification.

In [ ]:
# Function for training the classifier
def trainData():
    # Determine the feature sets
    featureSets = [(extractFeatures(tweetTerm), relevance) for (tweetTerm, relevance) in labeledTweets]
    # Randomize the data
    random.shuffle(labeledNames)

    # Establish the training and dev data sets
    trainSet, devSet = featureSets[500:], featuresSets[:500] #before and after 500

    # Train the Naive Bayes (NB) classifier
    classifierNB = nltk.NaiveBayesClassifier.train(trainSet)
# End trainData

In [ ]:
# The main method
def main():
    # Set the output file path
    txtFilePath = current_dir + '/OutputTweets.txt'
    # Create object for writting to a text file
    tweetTxtFile = open(txtFilePath, "w")
    
    # Request user input of the file name of the data to be processed
    inputFile = raw_input("Enter csv file name...\nEx: cleaned_geo_tweets_Apr_12_to_22")

    # Iterate through the Twitter data csv files by tweet text
    with open(current_dir + '/../' + inputFile + '.csv') as csvfile:  
        tweetIt = csv.DictReader(csvfile)
        # Retrieve the strings of tweets
        for twitterData in tweetIt:
            # Convert tweets to lower case to pool words of the same spelling
            # Send the tweet text to the function for removing unncessary characters
            tweetText = cleanUpTweet(twitterData['tweet_text'])
            # Write the selected Twitter data, tweets, to the txt file
            tweetTxtFile.write(tweetText)
    # Close the file obj
    tweetTxtFile.close()
    # Organize the data by relevance according to keyword dictionary
    orgData(tweetTxtFile)
    # Use the defined function to extract relevant features (terms)
    extractFeatures(tweetTxtFile)
# End main

# Run the script via the main method
if __name__ == "__main__":
    main()
    
# End script